In [71]:

import pandas as pd
import numpy as np, seaborn as sns, matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

import eda
import present_value

%load_ext autoreload
%autoreload 2
%reload_ext autoreload


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [72]:
## FROM DATABASE
pv = present_value.PresentValue()
anual_increment = pv.fetch_incremento_from_database()

preproccesing = eda.EDA()
df_raw = preproccesing.assemble_projects_from_database()
df_vp = preproccesing.create_dataset(pv.present_value_costs)

c:\Code\Road-Project-Prediction\src\eda.py:147: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [58]:
df_raw[df_raw['CÓDIGO DEL PROYECTO']=='0654801'].loc[:, "1 - TRANSPORTE": "16 - DIRECCIÓN Y COORDINACIÓN"].head()

,1 - TRANSPORTE,2.1 - INFORMACIÓN GEOGRÁFICA,2.2 TRAZADO Y DISEÑO GEOMÉTRICO,2.3 - SEGURIDAD VIAL,2.4 - SISTEMAS INTELIGENTES,3.1 - GEOLOGÍA,3.2 - HIDROGEOLOGÍA,4 - SUELOS,5 - TALUDES,6 - PAVIMENTO,7 - SOCAVACIÓN,8 - ESTRUCTURAS,9 - TÚNELES,10 - URBANISMO Y PAISAJISMO,11 - PREDIAL,12 - IMPACTO AMBIENTAL,13 - CANTIDADES,14 - EVALUACIÓN SOCIOECONÓMICA,15 - OTROS - MANEJO DE REDES,16 - DIRECCIÓN Y COORDINACIÓN
26,0.0,0.0,2.851699e+08,3.065453e+08,77864916.72,198605538.6,0.0,2.588137e+08,3.099794e+08,1.384100e+08,6.474906e+08,4.486878e+08,0.0,2.333073e+08,0.0,0.0,40998694.62,0.0,439848969.7,3.440089e+08
27,0.0,0.0,2.851699e+08,3.065453e+08,77864916.72,198605538.6,0.0,2.588137e+08,3.099794e+08,1.384100e+08,6.474906e+08,4.486878e+08,0.0,2.333073e+08,0.0,0.0,40998694.62,0.0,439848969.7,3.440089e+08
28,0.0,0.0,2.851699e+08,3.065453e+08,77864916.72,198605538.6,0.0,2.588137e+08,3.099794e+08,1.384100e+08,6.474906e+08,4.486878e+08,0.0,2.333073e+08,0.0,0.0,40998694.62,0.0,439848969.7,3.440089e+08
29,0.0,0.0,2.851699e+08,3.065453e+08,77864916.72,198605538.6,0.0,2.588137e+08,3.099794e+08,1.384100e+08,6.474906e+08,4.486878e+08,0.0,2.333073e+08,0.0,0.0,40998694.62,0.0,439848969.7,3.440089e+08
30,0.0,0.0,2.851699e+08,3.065453e+08,77864916.72,198605538.6,0.0,2.588137e+08,3.099794e+08,1.384100e+08,6.474906e+08,4.486878e+08,0.0,2.333073e+08,0.0,0.0,40998694.62,0.0,439848969.7,3.440089e+08


In [60]:
# Calculate COSTO_UF_VP: total cost per functional unit (sum all item columns per row)
# df_vp['COSTO_UF_VP'] = df_vp.loc[:, "1 - TRANSPORTE": "16 - DIRECCIÓN Y COORDINACIÓN"].sum(axis=1)
df_vp[df_vp['CÓDIGO']=='0654801'].loc[:, "1 - TRANSPORTE": "16 - DIRECCIÓN Y COORDINACIÓN"].sum(axis=0)


1 - TRANSPORTE                     0.000000e+00
2.1 - INFORMACIÓN GEOGRÁFICA       0.000000e+00
2.2 TRAZADO Y DISEÑO GEOMÉTRICO    3.324713e+08
2.3 - SEGURIDAD VIAL               3.573922e+08
2.4 - SISTEMAS INTELIGENTES        9.078045e+07
3.1 - GEOLOGÍA                     2.315484e+08
3.2 - HIDROGEOLOGÍA                0.000000e+00
4 - SUELOS                         3.017433e+08
5 - TALUDES                        3.613959e+08
6 - PAVIMENTO                      1.613683e+08
7 - SOCAVACIÓN                     7.548905e+08
8 - ESTRUCTURAS                    5.231120e+08
9 - TÚNELES                        0.000000e+00
10 - URBANISMO Y PAISAJISMO        2.720062e+08
11 - PREDIAL                       0.000000e+00
12 - IMPACTO AMBIENTAL             0.000000e+00
13 - CANTIDADES                    1.099381e+09
14 - EVALUACIÓN SOCIOECONÓMICA     0.000000e+00
15 - OTROS - MANEJO DE REDES       5.128072e+08
16 - DIRECCIÓN Y COORDINACIÓN      4.010700e+08
dtype: float64

In [80]:
# Get project info from original database for AÑO INICIO and FASE
import sqlite3
from config import Config

df_vp['COSTO_UF_VP'] = df_vp.loc[:, "1 - TRANSPORTE": "16 - DIRECCIÓN Y COORDINACIÓN"].sum(axis=1)

conn = sqlite3.connect(Config.DATABASE)
df_proyectos = pd.read_sql_query("SELECT codigo, anio_inicio, fase FROM proyectos", conn)
conn.close()

# Count functional units per project
df_vp['UF_COUNT'] = df_vp.groupby('CÓDIGO')['CÓDIGO'].transform('count')

# Aggregate by project - properly sum all functional units
# Group by CÓDIGO and aggregate different columns appropriately
df_project = df_vp.groupby('CÓDIGO').agg({
    'NOMBRE DEL PROYECTO': 'first',  # Take first (same for all rows)
    'ALCANCE': 'first',              # Take first (same for all rows)
    'LONGITUD KM': 'sum',            # Sum all functional units' lengths
    'COSTO_UF_VP': 'sum',            # SUM all functional units' costs (THIS IS KEY!)
    'UF_COUNT': 'first'              # Take first (same for all rows)
}).reset_index()

# Rename the aggregated cost column
df_project = df_project.rename(columns={'COSTO_UF_VP': 'COSTO_TOTAL_VP'})

# Merge with database info to get AÑO INICIO and FASE
df_project = df_project.merge(df_proyectos, left_on='CÓDIGO', right_on='codigo', how='left')

# Rename columns for clarity
df_project = df_project.rename(columns={
    'UF_COUNT': 'UNIDADES_FUNCIONALES',
    'anio_inicio': 'AÑO INICIO',
    'fase': 'FASE'
})

# Drop duplicate codigo column
df_project = df_project.drop(columns=['codigo'])

# Format cost values for display (in millions)
df_project['COSTO_MILLONES'] = df_project['COSTO_TOTAL_VP'] / 1_000_000

# Build hover_data dictionary
hover_dict = {
    'NOMBRE DEL PROYECTO': True,
    'CÓDIGO': True,
    'AÑO INICIO': True,
    'FASE': True,
    'ALCANCE': True,
    'UNIDADES_FUNCIONALES': True,
    'LONGITUD KM': ':.2f',
    'COSTO_MILLONES': ':.2f',
    'COSTO_TOTAL_VP': ':,.0f'
}

# Create the plot
fig = px.scatter(
    df_project,
    x='LONGITUD KM',
    y='COSTO_MILLONES',
    color='CÓDIGO',
    hover_data=hover_dict,
    labels={
        'LONGITUD KM': 'Longitud del Proyecto (km)',
        'COSTO_MILLONES': 'Valor Presente de la Causación de Personal (Millones COP)',
        'NOMBRE DEL PROYECTO': 'Nombre del Proyecto',
        'CÓDIGO': 'Código del Proyecto',
        'AÑO INICIO': 'Año de Inicio',
        'FASE': 'Fase del Proyecto',
        'ALCANCE': 'Alcance',
        'UNIDADES_FUNCIONALES': 'N° Unidades Funcionales',
        'COSTO_TOTAL_VP': 'Costo Total VP (COP)'
    },
    title='<b>Valor Presente de la Causación de Personal en Proyectos Viales</b><br><sub>Relación entre Longitud del Proyecto y Causación Total</sub>',
    template='plotly_white',
    size='COSTO_MILLONES',
    size_max=40,
    color_discrete_sequence=px.colors.qualitative.Set2
)

# Enhance the visual design
fig.update_traces(
    marker=dict(
        line=dict(width=1.5, color='white'),
        opacity=0.85,
        sizemin=8
    ),
    textposition='top center'
)

# Update layout for executive presentation
fig.update_layout(
    font=dict(family="Arial, sans-serif", size=12, color="#2c3e50"),
    title=dict(
        font=dict(size=18, color="#1a252f", family="Arial Black"),
        x=0.5,
        xanchor='center'
    ),
    xaxis=dict(
        showgrid=True,
        gridcolor='#ecf0f1',
        zeroline=False,
        title_font=dict(size=14, color="#34495e", family="Arial"),
        tickfont=dict(size=11)
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='#ecf0f1',
        zeroline=False,
        title_font=dict(size=14, color="#34495e", family="Arial"),
        tickfont=dict(size=11)
    ),
    plot_bgcolor='white',
    paper_bgcolor='white',
    hovermode='closest',
    legend=dict(
        title=dict(text='<b>Código Proyecto</b>', font=dict(size=12)),
        orientation="v",
        yanchor="top",
        y=1,
        xanchor="left",
        x=1.02,
        bgcolor="rgba(255, 255, 255, 0.9)",
        bordercolor="#bdc3c7",
        borderwidth=1
    ),
    margin=dict(l=80, r=200, t=100, b=80),
    height=600,
    width=1200
)

# Add a trend line for better insights
from sklearn.linear_model import LinearRegression
X = df_project[['LONGITUD KM']].values
y = df_project['COSTO_MILLONES'].values
model = LinearRegression()
model.fit(X, y)
y_pred = model.predict(X)

# Sort for line plot
sort_idx = X.flatten().argsort()
fig.add_trace(
    go.Scatter(
        x=X[sort_idx].flatten(),
        y=y_pred[sort_idx],
        mode='lines',
        name='Tendencia Lineal',
        line=dict(color='#e74c3c', width=2, dash='dash'),
        showlegend=True,
        hovertemplate='Línea de Tendencia<extra></extra>'
    )
)

fig.show()

In [74]:
pv.present_value(3729730912.08, 2023, 2025)

4348385108.197621